In [16]:
from collections import defaultdict
import qiskit
from qiskit.circuit.library import XGate
from qiskit.providers.aer import PulseSimulator
from qiskit.providers.aer.pulse import PulseSystemModel
from qiskit.providers.aer.pulse.hamiltonian_model import HamiltonianModel
from qiskit.providers.aer.pulse.qobj.opparse import HamiltonianParser
from qutip import identity, Qobj, sigmax
import numpy as np

from src.prefactor_parsing import prefactor_parser
from qiskit import IBMQ, pulse, QuantumCircuit, QuantumRegister, ClassicalRegister, assemble
from src.prefactor_parsing import prefactor_parser
from src.qoc_instruction_schedule_map import QOCInstructionScheduleMap
from qiskit.test.mock import FakeArmonk
from src.qutip_helper import convert_qutip_ham
from src.QutipOptimizer import QutipOptimizer

In [17]:
IBMQ.load_account()

# provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
backend = provider.get_backend('ibmq_armonk')
backend = FakeArmonk()

ibmqfactory.load_account:WARNING:2020-08-07 17:28:26,056: Credentials are already in use. The existing account in the session will be replaced.


In [18]:
omegad0 = getattr(backend.configuration(), 'hamiltonian')['vars']['omegad0']
omegad0 = omegad0 * .5 * 1e9

omegad0 = 31919806.545849085

getattr(backend.configuration(), 'hamiltonian')['vars']['omegad0'] = omegad0
freq_est = 4.97445401836326e9
backend.configuration().qubit_freq_est=[freq_est]
getattr(backend.configuration(), 'hamiltonian')['vars']['wq0'] = 2*np.pi*freq_est

In [19]:
hamiltonian = convert_qutip_ham(backend)
hamiltonian

{'H_c': {'D0': Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
  Qobj data =
  [[       0.         31919806.54584908]
   [31919806.54584908        0.        ]]},
 'iH_c': {'D0': Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
  Qobj data =
  [[0.       +0.j         0.+31919806.54584908j]
   [0.+31919806.54584908j 0.       +0.j        ]]},
 'H_d': Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
 Qobj data =
 [[0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 3.12554164e+10]]}

In [20]:
grape_optimizer = QutipOptimizer(backend, n_ts=1600)

In [21]:
result = grape_optimizer.run_optimizer(unitary=Qobj(XGate().to_matrix()))

[-0.7         0.42361043]
[0.7        0.54042808]


In [22]:
grape_optimizer.hamiltonian
grape_optimizer.controls

[Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
 Qobj data =
 [[       0.         31919806.54584908]
  [31919806.54584908        0.        ]],
 Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
 Qobj data =
 [[0.       +0.j         0.+31919806.54584908j]
  [0.+31919806.54584908j 0.       +0.j        ]]]